## Step 1 Generate PepPI Interaction

In [2]:
import pandas as pd
import numpy as np
import os


In [ ]:
import pandas as pd
import numpy as np
import os

def check_abnormal_aa(peptide_seq):
    len_seq = len(peptide_seq)
    cnt = 0
    standard_aa = ['G','A','P','V','L','I','M','F','Y','W','S','T','C','N','Q','K','H','R','D','E']
    for i in peptide_seq:
        if i in standard_aa :
            cnt = cnt+1
    score = float(cnt)/len_seq
    return score

def lower_chain(input_str):
    chain_list = list(input_str)
    output_list = []

    for item in chain_list:
        if item.isalpha() :
            a=item.lower()
        else :
            a=item
        output_list.append(a)
    output_str = ''.join(output_list)
    return output_str


# Step 0: parse the fasta file downloaded from the RCSB PDB
# INPUT : pdb_seqres.txt
# OUTPUT: pdb_pep_chain, pdbid_all_fasta
raw_str=''
print("Processing pdb_seqres")
with open('E:/example_data_feature/data_prepare/pdb_seqres2.txt','r') as f:
    for line in f.readlines():
        raw_str = raw_str+line.replace('\n','###')
raw_list = raw_str.split('>')
del raw_list[0]

PDB_id_lst = [x.split('_')[0] for x in raw_list]
PDB_chain_lst = [x.split('_')[1].split(' ')[0].lower() for x in raw_list]
PDB_type_lst = [x.split('mol:')[1].split(' ')[0] for x in raw_list]
PDB_seq_lst = [x.split('###')[1] for x in raw_list]
PDB_seq_len_lst = [len(x) for x in PDB_seq_lst]
df_fasta_raw =pd.DataFrame(list(zip(PDB_type_lst, PDB_seq_len_lst,PDB_seq_lst,PDB_id_lst,PDB_chain_lst)),\
                       columns=['PDB_type','PDB_seq_len','PDB_seq','PDB_id','chain'])
df_fasta = df_fasta_raw[(df_fasta_raw.PDB_seq_len<=50)&(df_fasta_raw.PDB_type=='protein')]
df_fasta_raw.to_csv('pdbid_all_fasta_all', encoding='utf-8', index=False, sep = '\t')
df_fasta.to_csv('pdb_pep_chain_all', encoding='utf-8', index=False, sep = '\t')

print('Step 0 is finished by generating two files : pdb_pep_chain & pdbid_all_fasta!')
# Step1 : Load all PDB ids that might contain peptide interaction and plip prediction results
# INPUT : pdb_pep_chain from Step 0 & analyzed file generated by PLIP (placed under ./peptide_result/ ,details in Data_construction.txt)
# OUTPUT: plip_predict_result

def load(pdb_pep_dataset,plip_result_filename): #pdb_pep_chain   #plip_predict_result
	df_fasta_pep = pd.read_csv(pdb_pep_dataset,sep='\t',header = 0) 
	df_fasta_pep=df_fasta_pep.reset_index(drop=True)
	df_predict = pd.DataFrame(columns=['pdb_id','pep_chain','predicted_chain'])

	for i in range(df_fasta_pep.shape[0]):
		pdb_id = df_fasta_pep['PDB_id'][i]
		chain = df_fasta_pep['chain'][i]
		print("PDB_ID:", pdb_id)
# 		os.system(f'python D:/Anaconda3/Lib/site-packages/plip-2.2.2-py3.8.egg/plip/plipcmd.py -i {pdb_id} --peptide {chain.upper()}  -vto E:/plip/outputresults/ --name {pdb_id}_{chain}_result')
		result_file_name = 'E:/plip/outputresults/'+pdb_id + '_'+chain+'_result.txt'
	    # print(result_file_name)
		try:
			for line in open(result_file_name):
				if line.startswith('Interacting chain(s):'):
					df_predict.loc[i] = [pdb_id,chain, str(line).replace('\n','')\
	                                     .replace('\r','')\
	                                     .replace('Interacting chain(s):','')
	                                     .lower()]
			if i % 5000 == 0:
				print('already finished files',i)
		except :
	        #print('find no file for',pdb_id)
	        #print(i,pdb_id,line)
			pass
	print('finish loading!')
	print('-----------------------------------------------------')
	print(df_predict)
	df_predict['predicted_chain_num'] = df_predict.predicted_chain.apply(lambda x :len(x.replace(' ','')) )
	df_predict = df_predict.loc[df_predict.predicted_chain_num>0]
	df_predict = df_predict.drop('predicted_chain_num',axis = 1)
	df_predict['predicted_chain'] = df_predict.predicted_chain.apply(lambda x :\
	                                      x.replace(' ',''))
	df_predict['pep_chain'] = df_predict.pep_chain.apply(lambda x :\
	                                      x.replace(' ',''))
	df_predict = df_predict.reset_index(drop=True)
	print('finish removing PDB ids without any interaction')
	print('-----------------------------------------------------')
	df_predict.predicted_chain = df_predict.predicted_chain.apply(lambda x: x.split(','))
	lst_col = 'predicted_chain'
	df1 = pd.DataFrame({col:np.repeat(df_predict[col].values, df_predict[lst_col].str.len())
	              for col in df_predict.columns.difference([lst_col])
	            }).assign(**{lst_col:np.concatenate(df_predict[lst_col].values)})[df_predict.columns.tolist()]
	df_predict = df1
	# save organized data formatted like (pdb,pep_chain,predicted_prot_chain)
	file_name = plip_result_filename
	df_predict.to_csv(file_name, encoding='utf-8', index=False, sep='\t')
	print('finish exploding comma-seperated predicted chain, successfully saved records:',df_predict.shape[0])

	print('Step 1 is finished by generating the PLIP prediction file : plip_predict_result. ')

	return df_predict

# Step 2: Get fasta sequence of the predicted interacting chains
# INPUT:  pdbid_all_fasta from Step 0
# OUTPUT: -
def load_all_fasta(all_fasta_file,input_dataset): # pdbid_all_fasta # df_predict
	df_fasta = pd.read_csv(all_fasta_file,sep = '\t', header = 0)
	df_fasta_protein = df_fasta.loc[df_fasta.PDB_type=='protein']

	# df_fasta_protein['PDB_id'] = df_fasta_protein.PDB_id_chain.apply(lambda x: x.split('_')[0])
	# df_fasta_protein['chain'] = df_fasta_protein.PDB_id_chain.apply(lambda x: x.split('_')[1].lower())
	df_fasta_vocabulary = df_fasta_protein[['PDB_id','chain','PDB_seq']]

	df_predict_det = pd.merge(input_dataset,df_fasta_vocabulary,how='left',\
	                         left_on = ['pdb_id','pep_chain'],right_on = ['PDB_id','chain'])

	df_predict_det1 = pd.merge(df_predict_det,df_fasta_vocabulary,how='left',\
	                         left_on = ['pdb_id','predicted_chain'],right_on = ['PDB_id','chain'])
	df_predict_det1 =df_predict_det1.drop(['PDB_id_x','chain_x','PDB_id_y','chain_y'],axis =1)
	df_predict_det1.columns = ['pdb_id','pep_chain','predicted_chain','pep_seq','prot_seq']
	print(df_predict_det1)    
	df_predict_det1['pep_seq_len'] = df_predict_det1.pep_seq.apply(lambda x: len(x))
	df_predict_det1['prot_seq_len'] = df_predict_det1.prot_seq.apply(lambda x: len(str(x)))
    

	# check sequence length(peptide<=50 & protein >50)
	df_predict_det1 = df_predict_det1.loc[(df_predict_det1.pep_seq_len <= 50) & (df_predict_det1.prot_seq_len > 50)]


	# remove records with more than 20% AA is abnormal
	df_predict_det1['peptide_seq_score'] = df_predict_det1.pep_seq.apply(lambda x: check_abnormal_aa(x))
	df_predict_det1 = df_predict_det1[df_predict_det1.peptide_seq_score >= 0.8]

	print('finish removing sequences without too many non-standard residues')
	print('-----------------------------------------------------')


	return df_predict_det1

# Step 3: Map Uniprot ID for PDB complex by protein-chain & PDB id
# INPUT: data from Step 2 & pdb_chain_uniprot.tsv from SIFT
# OUTPUT: UniProt_ID_list ( all IDs are the searching query on https://www.uniprot.org/uploadlists/ for unified sequence)
def map_uniprot_chain(input_dataset,pdb_chain_uniprot_file): #df_predict_det1 #pdb_chain_uniprot.tsv
	df_sifts = pd.read_csv(pdb_chain_uniprot_file, sep = '\t', header = 0)
	df_sifts = df_sifts[['PDB','CHAIN','SP_PRIMARY']]
	df_sifts_keep = df_sifts[df_sifts['CHAIN'] !=  df_sifts['CHAIN']]
	df_sifts = df_sifts[df_sifts['CHAIN'] == df_sifts['CHAIN']]
	df_sifts['CHAIN'] = df_sifts.CHAIN.apply(lambda x: lower_chain(x))

	df_predict_det2 = pd.merge(input_dataset,df_sifts, how = 'left', \
                              left_on = ['pdb_id','predicted_chain'],right_on = ['PDB','CHAIN'])
	df_predict_det2 = df_predict_det2.drop(['PDB','CHAIN'],axis = 1)


	# subset records that don't have a matched protein chain Uniprot
	df_predict_det2_no_uni = df_predict_det2[df_predict_det2.SP_PRIMARY != df_predict_det2.SP_PRIMARY]
	df_predict_det2_no_uni = df_predict_det2_no_uni.reset_index(drop = True)
	
	df_predict_det2_no_uni = df_predict_det2_no_uni.drop(['prot_seq_len','peptide_seq_score'],axis = 1)
	df_predict_det2_no_uni = df_predict_det2_no_uni[['pdb_id','pep_chain','predicted_chain','pep_seq',\
	                                 'pep_seq_len','SP_PRIMARY','prot_seq']]
	df_predict_det2_no_uni.rename(columns = {'prot_seq': 'Sequence'}, inplace=True)

	# focus on records with Uniprot Ids
	df_predict_det3 = df_predict_det2[df_predict_det2.SP_PRIMARY == df_predict_det2.SP_PRIMARY]

	# save matched uniprot ID for retrieving information from Uniprot Website
	df_uni_id = df_predict_det3[['SP_PRIMARY']]
	df_uni_id.drop_duplicates(inplace = True)
	file_name = pdb_chain_uniprot_file
	df_uni_id.to_csv('pdb_chain_uniprot_all.tsv', encoding = 'utf-8', index = False, sep = '\t')

	return df_predict_det2_no_uni,df_predict_det3


# Step 4: Load Uniport sequences and family information & filter out MHC families
# INPUT: the data from Step 3 & uniprot2seq from UniProt Website (a tab separated file with fields including Uniprot_id,Uniprot Sequence,Protein_name,Protein_families)
# OUTPUT: interacted peptide-protein pairs from PDB (a '#' separated file with fields including pdb_id,pep_chain,prot_chain,pep_seq,Uniprot_id,prot_seq,protein_families)

def load_uni_seq(df_predict_det2_no_uni, input_dataset,uniprot2seq_file):

	df_uni2seq = pd.read_csv(uniprot2seq_file,sep = '\t',header = 0)
	# df_uni2seq = df_uni2seq.drop('uniprot',axis = 1)
	df_uni2seq = df_uni2seq.drop_duplicates(['Uniprot_id','Sequence'],keep = 'first')
	df_uni2seq = df_uni2seq.fillna('Unknown_from_uniprot')

	# join by uniprot id
	df_predict_det4 = pd.merge(input_dataset,df_uni2seq,how = 'left',left_on = ['SP_PRIMARY'],right_on = ['Uniprot_id'])
	df_predict_det4 = df_predict_det4.drop(['Uniprot_id','Protein_name','prot_seq','prot_seq_len','peptide_seq_score'],axis = 1)
	df_predict_det4 = df_predict_det4.drop_duplicates(['pdb_id','pep_seq','SP_PRIMARY','Sequence'],keep = 'first')
	print(df_predict_det4)

	# filter out MHC
	df_predict_det4["MHC_flag"] = df_predict_det4.Protein_families.apply(lambda x: x.lower().find('mhc'))
	df_mhc = df_predict_det4.loc[df_predict_det4.MHC_flag!=-1][["pdb_id","Protein_families"]]
	df_mhc.columns = ['pdb_id_mhc','prot_family_mhc']

	# join by  PDB id only(if a pdb contains mhc proteins,remove all records of the PDB id)
	df_predict_det5 = pd.merge(df_predict_det4, df_mhc,left_on = ['pdb_id'], right_on = ['pdb_id_mhc'], how='left')
	df_predict_det5 = df_predict_det5.loc[df_predict_det5.pdb_id_mhc!=df_predict_det5.pdb_id_mhc]
	df_predict_det5 = df_predict_det5.drop(['pdb_id_mhc','prot_family_mhc','MHC_flag'],axis =1)
	df_predict_det5.drop_duplicates(inplace=True)
	print(df_predict_det5)
	print(df_predict_det2_no_uni)
	if(len(df_predict_det2_no_uni)>0):
# 		df_predict_det2_no_uni = df_predict_det2_no_uni.drop(['prot_seq_len','peptide_seq_score'],axis = 1)
		df_predict_det2_no_uni = df_predict_det2_no_uni[['pdb_id','pep_chain','predicted_chain','pep_seq','pep_seq_len','SP_PRIMARY','Sequence']]
# 		df_predict_det2_no_uni.rename(columns={'prot_seq': 'Sequence'}, inplace=True)
		df_predict_det2_no_uni['Protein_families'] = pd.Series(['Unknown Uniprot_ids' for x in range(df_predict_det2_no_uni.shape[0])])
	df_predict_det6 = pd.concat([df_predict_det2_no_uni,df_predict_det5],ignore_index=True)
	df_predict_det6['plip_prot_chain'] = df_predict_det6.predicted_chain.apply(lambda x:\
	                                                              x.upper())
	df_predict_det6 = df_predict_det6.drop_duplicates(['pep_seq','Sequence'],keep='first')
	df_predict_det6 = df_predict_det6.reset_index(drop=True) #8184
	df_predict_det6['prot_seq_len'] = df_predict_det6.Sequence.apply(lambda x: len(str(x)))
	df_predict_det6 = df_predict_det6[df_predict_det6.prot_seq_len<=5000]

	df_pdb_pairs = df_predict_det6[['pdb_id','pep_chain','plip_prot_chain','pep_seq','SP_PRIMARY','Sequence','Protein_families']]
	df_pdb_pairs.columns = ['pdb_id','pep_chain','prot_chain','pep_seq','Uniprot_id','prot_seq','protein_families']
	file_name = 'train_pairs_pdb'
	df_pdb_pairs.to_csv(file_name, encoding = 'utf-8', index = False)
	print(df_pdb_pairs)

	return df_pdb_pairs


df_predict = load('pdb_pep_chain_all','plip_predict_result_all')
df_predict_det1 = load_all_fasta('pdbid_all_fasta_all',df_predict)
df_predict_det2_no_uni,df_predict_det3 = map_uniprot_chain(df_predict_det1,'E:/example_data_feature/data_prepare/pdb_chain_uniprot.tsv')
df_pdb_pairs = load_uni_seq(df_predict_det2_no_uni, df_predict_det3,'uniprot2seq2.tsv')


In [4]:
pep_pairs = pd.read_csv('E:/Projek pak Wisnu/Projek Peptide-Protein Interaction/train_pairs_pdb_modified.csv')
pep_pairs['pdb_chain'].nunique()

6989

In [ ]:
df_uni2seq = pd.read_csv('uniprot2seq.tsv',sep = '\t',header = 0)

In [ ]:
df_predict_det2_no_uni

## Step 2 Peptide and Protein Feature Extraction (Intrinsic Disorder, PSSM)

## Intrinsic Disorder Processing

In [77]:
import pandas as pd
import numpy as np
import pickle
# Load Intrinsic disorder
# dict : {sequence: Intrinsic Disorder Matrix}
# Intrinsic Disorder Matrix : (sequence length ,3) , last dimension :(long , short, ANCHOR score)

def extract_intrinsic_disorder(filename,ind):
    fasta_filename = filename+'.fasta'
    disorder_filename = filename+'_'+ind+'.result'
    raw_fasta_list = []
    with open('E:/example_data_feature/Intrinsic Disorder/' + fasta_filename,'r',encoding='utf-8-sig') as f:
        for line in f.readlines():
            line_list = line.strip()
            raw_fasta_list.append(line_list)

    fasta_id_list = [x for x in raw_fasta_list if x[0]=='>']
    fasta_sequence_list = [x for x in raw_fasta_list if x[0]!='>']
    fasta_seq_len_list = [len(x) for x in fasta_sequence_list]
    print(len(fasta_id_list),len(fasta_sequence_list),len(fasta_seq_len_list))

    fasta_dict = {}
    for i in range(len(fasta_id_list)):
        fasta_dict[fasta_id_list[i]]=(fasta_sequence_list[i],fasta_seq_len_list[i])
    # load protein intrinsic disorder result
    raw_result_list = []
    with open('E:/example_data_feature/Intrinsic Disorder/' + disorder_filename,'r',encoding='utf-8-sig') as f:
            for line in f.readlines():
                line_list = line.strip()
                if (len(line_list)>0 and line_list[0]!='#'):
                    raw_result_list.append(line_list)

    intrinsic_id_list = [x for x in raw_result_list if x[0]=='>']
    intrinsic_score_list = [x.split('\t') for x in raw_result_list if x[0]!='>']
    start_idx = 0
    raw_score_dict = {}
    for idx in range(len(intrinsic_id_list)):
        prot_id = intrinsic_id_list[idx]
        seq_len = fasta_dict[prot_id][1]
        end_idx = start_idx + seq_len
        individual_score_list = intrinsic_score_list[start_idx:end_idx]
        individual_score_list=[x[2:] for x in individual_score_list]
        individual_score_array = np.array(individual_score_list,dtype='float')
        raw_score_dict[prot_id] = individual_score_array
        start_idx = end_idx
    print(len(fasta_dict.keys()),len(raw_score_dict.keys()))
    return fasta_dict, raw_score_dict

# long & short
fasta_dict_long, raw_score_dict_long = extract_intrinsic_disorder('input4','long') # the input fasta file used in IUPred2A
fasta_dict_short, raw_score_dict_short = extract_intrinsic_disorder('input4','short')


Intrinsic_score_long = {}
for key in fasta_dict_long.keys():
    sequence = fasta_dict_long[key][0]
    id_fasta=key.split('>')[1]
    seq_len = fasta_dict_long[key][1]
    Intrinsic = raw_score_dict_long[key]
#     if Intrinsic.shape[0]!= seq_len:
#         print('Error!')
    Intrinsic_score_long[id_fasta] = Intrinsic

    
Intrinsic_score_short = {}
for key in fasta_dict_short.keys():
    sequence = fasta_dict_short[key][0]
    id_fasta=key.split('>')[1]
    seq_len = fasta_dict_short[key][1]
    Intrinsic = raw_score_dict_short[key]
#     if Intrinsic.shape[0]!= seq_len:
#         print('Error!')
    Intrinsic_score_short[id_fasta] = Intrinsic
   

Intrinsic_score = {}
for seq in Intrinsic_score_short.keys():
    long_Intrinsic = Intrinsic_score_long[seq]
    short_Intrinsic = Intrinsic_score_short[seq]
    concat_Intrinsic = np.column_stack((long_Intrinsic,short_Intrinsic))
    Intrinsic_score[seq] = np.column_stack((long_Intrinsic,short_Intrinsic))

# print(Intrinsic_score)
with open('E:/example_data_feature/Intrinsic Disorder/output_intrisic_dict4','wb') as f: # 'output_intrisic_dict' is the name of the output dict you like
    pickle.dump(Intrinsic_score,f)

8498 8498 8498
6989 6989
8498 8498 8498
6989 6989


In [78]:
(pd.DataFrame.from_dict(data=Intrinsic_score, orient='index')
   .to_csv('E:/output_intrisic_dict4.csv', header=False))

D:\Anaconda3\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


## Multiprocessing PSSM

In [4]:
# checking missing output pssm
from os import walk
_, _, filenames = next(walk('E:/example_data_feature/PSSM/output_protein'), (None, None, []))

In [5]:
list_file=[x.split('.')[0] for x in filenames]
list_file[0]

'A0MMD0_A'

In [6]:
pep_pairs[~pep_pairs['Uniprot_chain'].isin(list_file)]['Uniprot_chain'].nunique()

3362

In [ ]:
import multiprocess as mp
import os

def pssm_extract(Uniprot_chain):
    import os
    print(Uniprot_chain)
    os.popen(f'psiblast -query E:/example_data_feature/PSSM/input_protein/{Uniprot_chain}.fasta -db nr -out_ascii_pssm E:/example_data_feature/PSSM/output_protein/{Uniprot_chain}.pssm -remote').read()

with mp.Pool(os.cpu_count() - 2) as pool:
    pool.map(pssm_extract, pep_pairs[~pep_pairs['Uniprot_chain'].isin(list_file)]['Uniprot_chain'].unique())
    pool.close()

In [44]:
import numpy as np
### Load Protein PSSM Files (first change the value of protein_number)
# prot_pssm_dict : key is protein sequence, value is protein PSSM Matrix
prot_pssm_dict_all={}
pep_not_found = []
prot_pssm_dict={}
prot_pssm_arr = []
pep_list = pep_pairs['pdb_chain']
protein_num = 0 ### NEED TO BE CHANGED TO the total number of protein sequences
for i in range(len(pep_list)):
    try:
        filename_pssm = pep_list[i]+'.pssm' # need to name each individual fasta and pssm file with the same prefix
        filename_fasta = pep_list[i]+'.fasta'
        prot_key = pep_list[i]
        pssm_line_list= []
        print(filename_pssm)
        with open('E:/example_data_feature/PSSM/input2/'+str(filename_fasta),'r') as f: # directory to store fasta files (single file of each protein)
            for line in f.readlines():
                prot_seq = line.strip()

        with open('E:/example_data_feature/PSSM/kurang/'+str(filename_pssm),'r') as f:  # directory to store pssm files (single file of each protein)
            for line in f.readlines()[3:-4]:
                line_list = line.strip().split(' ')
                line_list = [x for x in line_list if x!=''][2:22]
                if (len(line_list)!=20 or line_list[0]=='-I'):
#                     print('Error line:')
#                     print(line_list)
                    continue
                line_list = [int(x) for x in line_list]
                pssm_line_list.append(line_list)
            pssm_array = np.array(pssm_line_list)
            if pssm_array.shape[1]!=20:
                print('Error!')
                print(filename_pssm)
            else:
                print(pssm_array)
                prot_pssm_dict_all[prot_key] = (prot_seq,pssm_array)
                prot_pssm_dict[prot_seq]=pssm_array
                prot_pssm_arr.append(pssm_array)
    except:
        print('Pep not found with ID:', pep_list[i])
        pep_not_found.append(pep_list[i])


2kj4_b.pssm
[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0]
 [-2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3]
 [-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3

2p8q_b.pssm
[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0]
 [-2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3]
 [-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3

[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0]
 [-2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3]
 [-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3

[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0]
 [-2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3]
 [-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3

7bwn_l.pssm
[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0]
 [-2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1]
 [-1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1]
 [-2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3]
 [-1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2]
 [-1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2]
 [-1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3

5ax3_b.pssm
[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0]
 [-2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3]
 [-1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2]
 [-2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1]
 [ 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3]
 [-2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3]
 [-1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3]
 [-1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2]
 [-1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1]]
5bn6_a.pssm
[[ 4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0]
 [-2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3]
 [ 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1]
 [-2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3]
 [-1  0  0  2 -4  2  5 -2  0 -

In [17]:
import pickle
with open('E:/example_data_feature/PSSM/output_pssm_dict_kurang','wb') as f:  # 'output_pssm_dict' is the name of the output dict you like
    pickle.dump(prot_pssm_dict_all,f)

import pandas as pd
(pd.DataFrame.from_dict(data=prot_pssm_dict_all, orient='index')
   .to_csv('E:/example_data_feature/PSSM/output_pssm_dict_kurang.csv', header=False))

In [27]:
pssm_df = pd.DataFrame()
pssm_df['peptide']=pep_pairs['pdb_chain'].unique()
pssm_df['pssm']=prot_pssm_arr

# Auto Covariance PSSM

In [63]:
def row_to_col(X):
    transform_mat = []
    for i in range(20):
        col = []
        for j in range(len(X)):
            col.append(X[j][i])
        transform_mat.append(col)
    transform_mat = np.array(transform_mat)
    return transform_mat

In [64]:
def lagged_auto_cov(Xi,t):
    """
    for series of values x_i, length N, compute empirical auto-cov with lag t
    defined: 1/(N-1) * \sum_{i=0}^{N-t} ( x_i - x_s ) * ( x_{i+t} - x_s )
    """
    N = len(Xi)
    # use sample mean estimate from whole series
    Xs = np.mean(Xi)

    # construct copies of series shifted relative to each other, 
    # with mean subtracted from values
    end_padded_series = np.zeros(N+t)
    end_padded_series[:N] = Xi - Xs
    start_padded_series = np.zeros(N+t)
    start_padded_series[t:] = Xi - Xs

    auto_cov = 1./(N-1) * np.sum( start_padded_series*end_padded_series )
    return auto_cov


In [66]:
pssm_complete['matrix'][0].shape

(23, 20)

In [68]:
pssm_ac_all = []
for k in range(len(pssm_complete['matrix'])):
    pssm_ac_row = []
    print(pssm_complete['pdb_chain'][k])
    if(pssm_complete['matrix'][k].shape[0]==1):
        pssm_ac_all.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    else:
        trans_mat = row_to_col(pssm_complete['matrix'][k])
        for l in range(trans_mat.shape[0]):
            ac = lagged_auto_cov(trans_mat[l],1)
            pssm_ac_row.append(ac)
        pssm_ac_all.append(pssm_ac_row)

1acy_p
1ai1_p
1qgc_5
1vrk_b
1qs7_b
1xcq_p
2b1a_p
3mlt_p
2b1h_p
3mly_p
2oqj_c
2osl_p
3bky_p
3pp4_p
2otk_c
3ifn_p
2g47_c
4mvi_b
4nge_b
2waq_p
3hkz_p
2pmz_p
2y5t_e
2y5t_f
3d0l_c
3drq_c
3d36_c
3drt_c
3efd_k
3gn8_c
1m2z_b
3h0t_c
4qae_p
3he4_b
3hr5_r
3hxg_c
3ixt_p
3mlr_p
6db5_p
3mls_p
3mlu_p
6mnq_p
6mnr_p
3mlv_p
3mlw_p
3mlx_p
3uji_p
6db6_p
6db7_p
6mns_p
1f58_p
3mlz_p
3o42_b
3qo0_c
3ujj_p
4jo2_p
3zrj_x
4au3_e
4au3_f
4bkl_e
4bkl_f
4dzv_b
4g6f_p
5iq7_p
5u3j_a
4jo1_p
5v6l_p
4jqi_v
4n8c_x
4p2o_p
4u1e_b
5bk0_e
6d01_i
6d11_e
6o23_e
5dlm_x
5drz_q
5epm_c
5fgb_f
5fgc_a
5iq9_c
5k5g_a
2g48_c
5kn5_c
5mu2_x
5mv3_n
5mub_x
5mub_h
5mv4_x
5ock_a
5u3k_p
5u3m_a
5u3n_a
5u3l_p
5u3o_a
5wn9_a
5wna_b
6cxg_a
6dc8_p
6dca_p
6dc9_p
6e4y_p
6gk8_i
6h06_i
6h0e_i
6ly5_i
2wsc_i
5l8r_i
6mtp_p
6mtt_p
6mv5_p
6n4i_e
6n4q_e
6ni2_v
6plh_c
6pnj_m
7lx0_v
6qph_j
6rhz_j
6rn2_s
5fn3_g
5oa1_4
6oab_h
6qs4_s
7a23_r
6xlz_p
6xmi_f
6z7w_i
6z7y_e
2wc0_c
3w12_a
6hn5_a
6z7w_j
6z7y_f
2g54_c
3w11_b
6hn5_b
6z7z_f
5kqv_b
6zzx_j
6zzy_j
7a48_b
7a4d_e

2lxp_b
2lxt_c
2lyd_b
2m04_b
2m0j_b
2m14_b
2m32_b
2m55_b
2m8s_b
2ma9_a
2mak_b
2mbh_b
2mfq_b
2mgu_m
2mjf_a
2mkc_b
2mkc_c
2mnj_a
2mp2_c
2mqs_b
2mqs_d
2mre_b
2msr_a
2mtp_b
2mtp_c
2mur_a
2mv7_b
2mw2_b
2mwn_a
2my2_b
2mzd_b
2mzp_i
2n01_a
2n0y_b
2n13_a
2n2h_a
2n4q_a
2n8j_b
2n8r_b
2nb1_b
2nl9_b
2v6q_b
2wh6_b
3fdl_b
4d2m_b
4uf3_b
4zie_c
5wos_b
2nla_b
2nmv_b
2nnu_b
2o01_j
2o01_i
3lw5_i
2o5g_b
2o60_b
2o8m_c
2ocv_a
2od8_b
2odb_b
2odg_c
2ofq_b
2oin_c
2p59_c
2qv1_c
2okr_c
2ot8_c
2ov2_i
2ovm_b
2p1l_b
2p4r_t
2p5b_i
2p8w_s
2pbd_v
2pg1_i
2pgq_a
2phe_c
2phg_b
2plx_b
2pon_a
2pqk_b
2pux_a
2qas_b
2qfa_c
2qiy_c
2qme_i
2qtv_d
2qxv_b
2r0l_b
2r28_c
2rdd_b
2rfd_c
2rfe_e
2riv_b
2rjf_b
2rky_b
2rkz_m
2roc_b
2rod_b
2roz_a
2rpq_b
2rqh_a
2rqw_b
2rr3_b
2rr9_c
2rsy_b
2ruk_a
2uwj_f
2uxn_e
2v1d_c
4gur_c
2v2f_a
2zc5_a
2v51_e
2v52_m
2v53_c
2v8c_c
2v8f_c
2vay_b
2vbc_b
2wzq_c
2vda_b
2vgo_c
2vm6_b
2vof_b
2vog_b
2voy_e
2voy_g
2voy_h
2vp7_b
2vpb_b
2vpe_b
2vu8_i
2vus_i
2vzd_c
5w93_d
2vzg_a
2vzi_a
2w2h_c
2w84_b
2wax_b
2wby_c
2wh0_q

6wpw_p
6wru_r
6wu9_6
6ww9_x
6wy6_c
6wzg_p
6x18_p
6x45_e
6x5g_b
6x5q_b
6xaw_b
6xfl_b
6xmn_b
6xqi_f
6xy4_b
6yac_i
6yhs_b
6ymx_u
6yn1_e
6yns_n
6yp7_i
6ypc_h
6z5r_c
6z5r_e
6zbi_b
6zn3_c
6zvi_p
6zvq_b
6zw0_c
7a23_s
7emf_b
7aam_c
7acv_c
7ad6_c
7ak9_c
7aos_c
7aos_d
7apo_c
7bk4_b
7ar7_v
7ate_d
7azx_c
7b2b_b
7bgi_i
7d0j_i
7bgi_j
7blz_m
7bqu_b
7bqv_b
7brg_l
7bri_l
7brj_l
7bv4_c
7bxt_k
7c9r_e
7c9r_r
7c9r_8
7c9r_0
7cg0_5
7cmz_b
7cna_b
7cnc_b
7cnu_c
7cnw_b
7co1_b
7cqh_a
7cqp_c
7cru_b
7czl_f
7czl_i
7ddq_n
7ddq_u
7ddq_a
7deg_c
7dr0_j
7drm_e
7ds2_c
7ds3_c
7ds6_c
7ds8_c
7dwq_w
7dwq_m
7dz9_e
7dz9_f
7e34_c
7e87_j
7e8e_k
7e8e_l
7e9k_c
7eda_m
7ek6_b
7ep7_b
7eqz_i
7ewj_c
7exe_c
7f16_p
7f7x_b
7f9y_c
7fbp_b
7fc0_a
7fim_p
7fiy_p
7fin_p
7jjl_b
7jjm_a
7jk7_a
7jqb_f
7jqd_b
7k48_e
7k95_b
7kdf_e
7ki1_p
7kl5_b
7ksq_m
7l1u_l
7len_c
7let_c
7lf4_d
7lew_b
7lll_p
7lly_p
7lo0_i
7lps_c
7lwh_b
7m5a_b
7m5b_b
7m5c_b
7md4_n
7mex_d
7mix_b
7mj5_a
7mla_b
7mqo_a
7mu2_b
7n40_c
7n8o_j
7ndx_b
7nmi_a
7np2_p
7nuf_c
7nxv_c
7o6n_c
7ogr_x

2m3m_b
2m41_a
2mc6_b
2mg5_b
2mho_b
2mip_e
2mjv_a
2mkp_i
2mkr_b
2mow_b
2mpm_b
2mps_b
6ijq_a
2mrk_b
3zmp_c
2ms4_b
2mwo_b
2mwp_b
2n1g_b
2n23_b
2n3a_a
2n3k_b
2pcu_b
2pdz_b
2peh_c
4oz1_c
2pem_r
2phk_b
2pie_f
2pkl_b
2pku_b
2pl9_d
2pld_b
2pq2_b
2pqw_b
2pr9_p
2puq_i
2pux_c
2pv1_b
2pv2_e
2pvc_d
3t6r_d
4qbr_p
5wxg_p
5y20_p
6j2p_e
2pvf_b
2q0n_b
2q3c_b
2q5y_b
2q8d_f
2q8y_b
2q9i_m
2qa9_i
2qbw_b
2qbx_d
2qfc_c
2qki_g
2ql5_g
2qll_b
2qn6_c
2qos_a
2qpy_b
2qqf_b
6rxm_g
2qt5_x
2r02_b
2r03_b
2r05_b
2r0y_b
2r3y_d
2zvl_u
3bfq_f
3bim_i
3bin_b
3l3g_c
3l3i_c
3l3j_c
3l3k_c
3l3q_b
3l3x_b
3l3z_b
3l41_b
3l6x_b
3l81_b
3lv3_c
3lw1_p
3m17_i
3m1b_i
3m4c_e
3m5m_c
3m5n_f
3m5o_c
3m61_p
3m7u_b
3ma3_b
3maz_b
3mev_c
3mhr_p
3ml4_e
3mmg_c
3mne_b
4qe6_b
5h1e_c
3mo8_b
3mp6_p
3uec_b
3mpj_y
3mqr_b
3mqs_d
3mt6_1
5dkp_o
3muk_d
3mul_d
3mxc_l
3mxy_l
3n2d_b
3n55_b
3n5e_d
3n5u_c
3n84_g
3n93_c
3n95_e
3n96_e
3n9q_c
3nco_d
3nco_e
3nf3_c
3nfl_e
3ngy_e
3nih_b
3nii_b
3nij_b
3nik_x
3nil_x
3nim_x
3nin_d
3njn_b
3nkx_p
3o8i_b
3nth_c
3nti_c
7cfd_i

5jiy_f
5jjm_m
5jjm_g
5jjy_b
5jjz_b
5jlb_b
5jme_f
5jqi_g
5jub_c
5jy0_b
5k18_f
5k58_l
5k6s_b
5k99_c
5k9s_b
5kds_g
5kjh_d
5kkl_d
5klr_a
5klt_a
5koa_d
5kvm_b
5kwn_u
5kyw_c
5kzt_d
5l23_b
5l7k_b
5l83_c
5l9v_c
5lb7_c
5leo_g
5nf0_f
7ok9_p
5lgp_e
5lgq_f
5lgr_e
5lgs_e
5lih_f
5ljn_c
5lm5_c
5lmf_c
5lmg_c
5lrg_f
5lrk_f
5ls6_q
5lso_c
5lsp_x
5lu1_c
5lu2_c
5lum_f
5lv0_c
5lvf_b
6qtc_b
5lvp_e
5lx2_b
5lxh_e
5ly1_e
5ly2_e
5m35_c
5m35_d
5m5g_e
5m5r_c
5m5s_e
5m5t_e
5m5u_e
5m5v_e
5m61_e
5m9d_b
5m9n_c
5m9o_b
5mav_g
5mbw_b
5mco_b
5mf9_b
5mgx_a
5mhc_p
5mk1_e
5mk3_e
5v3r_b
5mlu_m
5mlw_b
5mtw_e
5mxo_p
5my9_p
5n10_f
5n1w_c
5n22_e
5n5r_p
5n5t_p
5n70_b
5n7b_i
5n7g_c
5n7x_c
5n85_b
5n89_c
5n8b_e
5n8e_e
5n8j_p
5n8w_c
5n99_c
5nas_c
5nc7_k
5ncl_d
5ney_e
5nf0_e
5nfu_c
5ngq_e
5ni5_c
5nif_3
5nj9_e
5nja_e
5njc_e
5njc_f
5njf_e
5njj_e
5njx_b
5nkp_d
5nnc_c
5nne_c
5nng_b
5nnp_i
5no2_l
5nqa_g
5nqv_e
5ntj_b
5nwi_p
5nwj_p
5nxq_d
5o0e_c
5o45_b
5o4y_a
5o9s_c
5o9t_c
5o9u_c
5o9v_c
5oa1_q
5ods_e
5oj5_b
5ojo_c
5ojr_e
5ok3_d
5ok6_c
5onj_d

7nxe_c
7o1f_j
7o54_b
7ob5_p
7obc_b
7obg_b
7obk_b
7obs_b
7obx_b
7ofi_c
7ofk_c
7olg_c
7opm_b
7opm_c
7opw_b
7oun_b
7oxg_c
7oxh_b
7oxi_b
7oxk_b
7p00_p
7p0l_c
7p1c_b
7p93_b
7pc3_c
7pc4_c
7pc9_c
7pc5_b
7pc7_e
7pc8_c
7piu_p
7piv_p
7pul_p
7q50_b
7q51_b
7qa8_k
7qbs_b
7qcr_c
7qcs_c
7qct_c
7qip_c
7qql_f
7qqn_b
7qqy_b
7qto_c
7qtp_b
7r51_c
7r8w_b
7r8x_c
7rmi_s
7rn7_f
7rns_b
7rnu_b
7rnv_b
7rov_e
7rwa_y
7rxq_b
7ryc_l
7s2t_f
7s4q_f
7s8l_a
7s8m_a
7szi_d
7t10_p
7t11_p
7t6t_l
7t6v_l
7wvv_l
7tql_h
7txu_e
7uiq_c
7uso_f
7usp_f
7usq_f
7v1y_e
7vec_m
7vfl_d
7vfm_e
7vpw_b
7vv0_l
7vze_e
7w2z_l
7wvw_l
7yxw_d
2kj4_b
2l53_b
3he5_a
3o40_b
4dzu_b
4qoy_e
5ghw_p
6dlc_b
6dsl_a
6hqu_i
6j7w_c
6q53_b
6xp6_c
6yp7_l
7bee_c
7ci2_c
7dfa_v
7dfc_v
7dr0_i
7ewk_i
7nab_d
7x9e_e
1cu4_p
1f90_e
1u95_c
2a6d_p
2a6i_p
2f53_c
2fx9_p
2pye_c
2q3y_b
3lrh_b
3mob_p
3u0t_e
4bh8_p
4ris_p
4xcf_p
5ea0_p
5nme_c
5th2_e
5u1g_y
5wkf_c
6hqu_o
6n7j_c
6ncp_f
6s9l_c
6wl2_b
7bfi_f
7byd_h
7dnf_b
7mgv_v
7x5v_e
1abi_i
1aw8_a
1dev_b
1dzi_b
1ebd_c
1f0c_b
1f8v_d

In [70]:
pssm_ac_all = np.array(pssm_ac_all)
pssm_ac_all_df = pd.DataFrame([x for x in pssm_ac_all])
pssm_ac_all_df['pdb_chain']=pssm_complete['pdb_chain']
pssm_ac_all_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,pdb_chain
0,-0.352552,0.045455,-1.586527,-0.545540,-1.025262,-0.220399,-0.834679,-0.509452,-1.187919,-0.796958,...,-0.641433,0.448445,-0.659134,-0.075786,0.179326,0.801255,0.294209,-0.642378,-0.460646,1acy_p
1,-0.352552,0.045455,-1.586527,-0.545540,-1.025262,-0.220399,-0.834679,-0.509452,-1.187919,-0.796958,...,-0.641433,0.448445,-0.659134,-0.075786,0.179326,0.801255,0.294209,-0.642378,-0.460646,1ai1_p
2,-0.352552,0.045455,-1.586527,-0.545540,-1.025262,-0.220399,-0.834679,-0.509452,-1.187919,-0.796958,...,-0.641433,0.448445,-0.659134,-0.075786,0.179326,0.801255,0.294209,-0.642378,-0.460646,1qgc_5
3,-0.433684,0.105263,-1.892632,-0.434342,-1.187368,-0.194868,-0.710000,-0.555789,-1.481711,-0.860658,...,-0.674342,0.509868,-0.573816,0.068289,0.139342,0.864605,-0.368947,-0.446842,-0.507368,1vrk_b
4,-0.433684,0.105263,-1.892632,-0.434342,-1.187368,-0.194868,-0.710000,-0.555789,-1.481711,-0.860658,...,-0.674342,0.509868,-0.573816,0.068289,0.139342,0.864605,-0.368947,-0.446842,-0.507368,1qs7_b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984,-1.197279,-0.394558,-3.935374,-5.292517,-5.217687,-0.833333,-1.965986,-1.673469,-0.513605,-1.336735,...,-0.727891,-0.860544,-3.030612,-0.333333,-0.156463,0.139456,-0.006803,-0.989796,-0.836735,7t6u_l
6985,-0.399964,-0.172840,-2.641794,-0.967320,-1.335694,-0.417393,-1.044118,-0.691176,-1.963689,-0.711511,...,-0.985476,0.616013,-0.733297,0.007444,0.036311,0.814633,-0.018155,-0.573529,-0.450073,7tbh_a
6986,-0.660911,-0.995290,-3.464678,-1.097724,-2.019231,-1.100471,-1.401884,-0.826923,-2.547488,-1.299843,...,-1.715856,0.888540,-1.442308,-0.244898,-0.222920,0.142465,0.062794,-0.728414,-0.709576,7wui_l
6987,-9.000000,0.000000,-9.000000,-9.000000,-2.250000,-0.250000,-1.000000,-0.250000,-1.000000,-1.000000,...,-0.250000,-1.000000,-0.250000,-0.250000,-0.250000,-0.250000,-0.250000,-0.250000,-2.250000,8gch_c


In [71]:
pssm_ac_all_df.to_csv('pssm_complete_ac.csv',index=False)

# Cross Covariance PSSM

In [72]:
def crosscovariance(Xi,Yi,k):
    N = np.size(Xi)
    Xs = np.average(Xi)
    Ys = np.average(Yi)
    crossCov = 0
    for i in np.arange(0, N-k):
        crossCov += ((Yi[i+k])-Xs)*(Xi[i]-Xs)
    return (1/(N-1))*crossCov

In [73]:
pssm_cc_all = []
for k in range(len(pssm_complete['matrix'])):
    print(pssm_complete['pdb_chain'][k])
    if(pssm_complete['matrix'][k].shape[0]==1):
        pssm_cc_all.append([0]*380)
    else:
        trans_mat = row_to_col(pssm_complete['matrix'][k])
        pssm_cc_row = []
        for l in range(trans_mat.shape[0]):
            for m in range(trans_mat.shape[0]):
                if (l==m):
                    continue
                else:
                    cc = crosscovariance(trans_mat[l],trans_mat[m],1)
                    pssm_cc_row.append(cc)
        pssm_cc_all.append(pssm_cc_row)

1acy_p
1ai1_p
1qgc_5
1vrk_b
1qs7_b
1xcq_p
2b1a_p
3mlt_p
2b1h_p
3mly_p
2oqj_c
2osl_p
3bky_p
3pp4_p
2otk_c
3ifn_p
2g47_c
4mvi_b
4nge_b
2waq_p
3hkz_p
2pmz_p
2y5t_e
2y5t_f
3d0l_c
3drq_c
3d36_c
3drt_c
3efd_k
3gn8_c
1m2z_b
3h0t_c
4qae_p
3he4_b
3hr5_r
3hxg_c
3ixt_p
3mlr_p
6db5_p
3mls_p
3mlu_p
6mnq_p
6mnr_p
3mlv_p
3mlw_p
3mlx_p
3uji_p
6db6_p
6db7_p
6mns_p
1f58_p
3mlz_p
3o42_b
3qo0_c
3ujj_p
4jo2_p
3zrj_x
4au3_e
4au3_f
4bkl_e
4bkl_f
4dzv_b
4g6f_p
5iq7_p
5u3j_a
4jo1_p
5v6l_p
4jqi_v
4n8c_x
4p2o_p
4u1e_b
5bk0_e
6d01_i
6d11_e
6o23_e
5dlm_x
5drz_q
5epm_c
5fgb_f
5fgc_a
5iq9_c
5k5g_a
2g48_c
5kn5_c
5mu2_x
5mv3_n
5mub_x
5mub_h
5mv4_x
5ock_a
5u3k_p
5u3m_a
5u3n_a
5u3l_p
5u3o_a
5wn9_a
5wna_b
6cxg_a
6dc8_p
6dca_p
6dc9_p
6e4y_p
6gk8_i
6h06_i
6h0e_i
6ly5_i
2wsc_i
5l8r_i
6mtp_p
6mtt_p
6mv5_p
6n4i_e
6n4q_e
6ni2_v
6plh_c
6pnj_m
7lx0_v
6qph_j
6rhz_j
6rn2_s
5fn3_g
5oa1_4
6oab_h
6qs4_s
7a23_r
6xlz_p
6xmi_f
6z7w_i
6z7y_e
2wc0_c
3w12_a
6hn5_a
6z7w_j
6z7y_f
2g54_c
3w11_b
6hn5_b
6z7z_f
5kqv_b
6zzx_j
6zzy_j
7a48_b
7a4d_e

2jzi_b
2k03_b
2k2i_b
2k2u_b
2k3u_b
2k42_b
2k7w_b
2k8f_b
2k9u_b
2ka6_b
2kbw_b
2kc8_b
2kdu_b
2kft_b
2kgb_i
2khs_b
2kje_b
2kne_b
2kqs_b
2kri_b
2kt5_b
2ktf_b
2kwo_b
2wp2_p
5fa5_c
6mlc_e
2kwu_a
2kwv_a
2kxc_b
2lnh_c
2kxh_b
2kxq_b
2kxw_b
2l0f_b
2l14_b
2l1c_b
2l1l_a
2l1w_b
2l2i_b
2l35_b
2l6y_a
2l7l_b
2l9s_a
2le8_b
2leh_b
2li5_b
2lkm_a
2lnk_c
2lp0_b
2lpb_b
2lqc_b
2lr1_b
2lsj_b
2lxp_b
2lxt_c
2lyd_b
2m04_b
2m0j_b
2m14_b
2m32_b
2m55_b
2m8s_b
2ma9_a
2mak_b
2mbh_b
2mfq_b
2mgu_m
2mjf_a
2mkc_b
2mkc_c
2mnj_a
2mp2_c
2mqs_b
2mqs_d
2mre_b
2msr_a
2mtp_b
2mtp_c
2mur_a
2mv7_b
2mw2_b
2mwn_a
2my2_b
2mzd_b
2mzp_i
2n01_a
2n0y_b
2n13_a
2n2h_a
2n4q_a
2n8j_b
2n8r_b
2nb1_b
2nl9_b
2v6q_b
2wh6_b
3fdl_b
4d2m_b
4uf3_b
4zie_c
5wos_b
2nla_b
2nmv_b
2nnu_b
2o01_j
2o01_i
3lw5_i
2o5g_b
2o60_b
2o8m_c
2ocv_a
2od8_b
2odb_b
2odg_c
2ofq_b
2oin_c
2p59_c
2qv1_c
2okr_c
2ot8_c
2ov2_i
2ovm_b
2p1l_b
2p4r_t
2p5b_i
2p8w_s
2pbd_v
2pg1_i
2pgq_a
2phe_c
2phg_b
2plx_b
2pon_a
2pqk_b
2pux_a
2qas_b
2qfa_c
2qiy_c
2qme_i
2qtv_d
2qxv_b
2r0l_b
2r28_c

6hgd_b
6hgf_b
6hik_l
6hkp_s
6hl5_s
6hl6_s
6hlw_b
6hm8_b
6hmv_b
6hom_b
6hu9_h
6hu9_t
6hum_q
6hum_p
6i0y_7
6i3z_b
6i7q_h
6i85_b
6idx_c
6ifc_b
6igz_i
6ijj_j
6ijo_j
6ild_c
6iu7_b
6iua_b
6iuh_c
6iui_c
6iur_c
6iwa_a
6ixp_b
6ixq_b
6ixr_b
6j08_d
6j0w_c
6j4v_c
6j5i_i1
6j5i_i2
6j8f_a
6j8o_a
6j8y_d
6j9l_e
6jat_b
6jbk_b
6jcu_b
6jh8_b
6jh9_b
6jjw_u
6jjx_d
6jjy_u
6jle_e
6jmt_i
6jmu_c
6jnf_d
6jwj_c
6jx3_a
6jxv_b
6jxw_b
6k61_j
6kac_i
6khi_p
6l7o_p
6khi_q
6kn7_r
6kpd_b
6ku0_b
6kxx_b
6l35_j
7ksq_j
6l35_i
7ksq_i
6l4z_g
6l7o_q
6l8g_a
6l9f_c
6los_f
6los_g
6los_h
6lpb_p
6lph_b
6lum_e
6ly5_m
6m64_b
6m7a_d
6m7b_c
6m7y_c
6m8r_k
6m90_c
6m91_c
6mbb_b
6mbc_b
6mbd_c
6mbe_b
6mf6_d
6mgn_b
6mhf_c
6mjc_b
6mje_b
6ml1_g
6ms4_b
6msl_c
6msu_c
6mue_b
6muo_k
6mup_k
6n5w_b
6nbf_p
6nfj_c
6niy_p
6nro_d
6ntx_c
6nuc_c
6nuf_c
6nyx_g
6o09_i
6o40_b
6o8c_d
6ocf_b
6ocg_b
6och_b
6oi3_b
6oj7_c
6oni_d
6wms_e
6oyl_b
6p6e_b
6p9x_p
6p9y_p
6pb0_u
6pb1_u
6pnj_j
6prl_b
6psd_f
6pse_c
6pxu_c
6pzv_d
6q00_b
6q01_c
6q67_b
6qfm_b
6qg8_b
6qld_c
6qmp

2bz8_c
2j6f_c
2bzk_a
2c1j_c
2c23_p
2c2l_e
6fdp_b
2c3i_a
2c5v_f
2c5w_a
2c77_b
2c9f_s
2c9t_k
2cbl_b
3zni_b
2cch_e
2ce8_x
2ce9_x
2ci9_l
2cia_l
2cnm_d
2cnx_p
3mp1_p
4q6f_f
2co2_b
2cv3_b
2cvy_b
2cwg_d
2czy_b
2d1x_p
2d5w_c
2d5w_d
2dew_a
2dex_a
2dey_a
2df6_c
2do2_p
2dp4_i
2dqk_p
2drk_b
2drm_e
2ds8_p
2duj_p
2dvq_p
5fwe_c
7m98_b
2dvs_p
2dwx_p
2dyh_b
2dze_x
2egn_b
2ejy_b
2emt_c
2eph_h
2f69_b
2fci_b
2ff3_c
2ff4_e
2ffd_g
2ffd_h
2fff_a
2ffu_p
2fge_d
2fka_b
2flu_p
2fmk_b
2fns_p
2fnx_p
2foj_b
2foo_b
2fop_b
2fp7_c
3u1i_e
2fts_p
2fvj_b
2fym_b
2fys_d
2g2f_c
2g30_p
2g30_s
2g35_b
2g5l_x
2g83_c
2g99_c
2gfa_c
2ke1_b
2mnz_b
2puy_e
3o37_e
3uii_p
5lug_f
5t8r_e
5yko_p
6ie6_p
7ef1_p
7klr_b
2ghq_c
2ght_c
2gns_p
2gph_b
2grl_e
2gs6_b
2gtk_b
2h1c_b
2h1u_p
2h2g_b
2h43_i
2h6f_p
2h6k_c
2jmj_p
2kgi_b
3lqi_r
3sou_d
4mzh_b
5f6k_m
5mr8_c
5yc3_p
5yvx_c
2h6p_c
2h9e_s
2h9m_b
2l3r_b
3me9_c
3o7a_b
4o62_d
4pl6_c
4tmp_b
5tdr_b
6ie4_b
2hdx_g
2hgt_j
2hjl_c
2hl3_c
2hlo_g
2hod_m
2hmh_b
2hpl_b
2hpz_b
2ht9_x
2hu2_b
2hug_b
2hwl_p
2i04_c

4yzh_b
4z09_c
4z0d_c
4z0y_e
4z2o_p
4z2p_p
4z33_c
4z5w_p
4z63_p
4z7i_c
5ab0_e
4z7n_g
4z88_m
4z89_a
4z8j_b
4z8m_c
4zdu_b
4zhb_b
4zhl_p
6a8n_p
4zhm_p
4zjx_b
4zko_p
4zko_q
4zks_p
4zny_b
4zo1_a
4zq0_e
4zrt_b
4ztd_d
4ztd_f
4zu1_a
4zxl_h
4zzj_b
5a29_b
5a2i_p
5a2l_p
5a2q_r
5a31_v
5a86_c
5ab2_c
5aer_b
5afg_b
5afw_b
5agu_c
5ah2_e
6dj8_c
6djk_b
6dly_c
6dm6_c
6p81_b
6pth_g
6ptr_x
6ptv_x
5ajn_p
5ajo_b
5am8_p
5amb_p
5awt_b
5awu_b
5axi_e
5az8_b
5azf_c
5e6o_e
5azg_c
5azt_c
7wgo_b
5b4w_g
5b56_c
5b5u_c
5b62_b
5b6c_b
5bjt_p
5bmm_c
5bnw_d
5vvx_b
5bpu_i
5bs2_r
5btv_p
5bwn_b
5bwo_b
5bxo_c
5bxu_b
5c0m_c
5c11_b
5c13_p
5c26_b
5c27_d
5c6v_e
5c7e_g
5c7f_e
5c7z_b
5cdw_d
5cfa_d
5ciu_c
5cj6_b
5co5_f
5cqx_c
5crw_b
5cve_d
5cxv_c
5czh_b
5czi_b
5d13_g
5d2a_d
5d6y_a
5d7e_c
5d7f_p
5d94_b
5d9g_c
5dah_c
5dai_b
5ddz_b
5de2_c
5df6_b
5dhf_d
5di7_c
5di8_c
5nsc_c
6ifj_c
5dif_d
5dir_e
6ryo_b
5diy_p
5vvu_b
5dms_b
5dpw_b
5drv_b
5dx1_f
5dxa_f
5dxe_c
5e0l_c
5e0m_c
5e0v_c
5e1b_d
5e1d_d
5e1m_d
5e1o_d
5e2a_d
6dub_e
5e33_b
5e3a_b
5e3c_b

6tjt_d
6tkk_b
6tno_b
6tnq_b
6tof_b
6tq0_b
6tqr_e
6trw_d
6tsc_b
6ttk_f
6twb_b
6twn_c
6twq_c
6twx_c
6twu_c
6twy_c
6tyt_b
6tyt_c
6tyu_b
6tyv_b
6u0o_d
6u22_c
6u2f_b
6u3i_b
6u4a_d
6u61_e
6u6k_c
6u6l_b
6u72_c
6u74_e
6u8g_e
6u8h_c
6u8i_b
6u8m_c
6ubh_e
6ugd_g
6uge_g
6ugm_r
6uh5_r
6uj8_c
6ulp_c
6ulq_d
6uls_b
6ulv_e
6uoe_p
6v2e_b
6v2h_b
6v3n_c
6v3x_b
6v3y_b
6v4e_c
6v4f_b
6v4g_b
6v4h_b
6v55_q
6v7o_c
6v7p_d
6v84_c
6vbx_b
6vdb_h
6vil_i
6vj9_b
6vp9_c
6vqp_q
6vxy_c
6w0l_p
6w9k_b
6w9m_b
6wau_g
6wav_e
6way_v
6wba_c
6wbb_c
6wbc_b
6wc6_c
6wgn_e
6whn_f
6who_f
6wi3_f
6wjq_c
6wlx_b
6wly_b
6wnx_c
6wqu_d
6wsj_i
6wzx_c
6wzz_b
6x23_b
6x2p_d
6x2r_d
6x2s_d
6x2u_d
6x2v_d
6x2w_d
6x2x_d
6x2y_d
6x5r_c
6x5v_b
6x5w_b
6x6m_b
6x6q_b
7ec6_d
6xa6_c
6xa8_c
6xar_c
6xbe_f
6xbf_f
6xfk_b
6xi2_h
6xib_i
6xid_i
6xnj_b
6xqi_h
6xro_b
6xrp_b
6xs5_b
6xs7_b
6xs8_b
6xs9_c
6xsa_b
6xte_b
6xtl_c
6xtm_c
6xvd_p
6xwd_p
6xxc_b
6xxs_c
6zbu_c
6xzz_b
6y0u_e
6y0w_c
6y0x_j
6y1d_b
6y1j_p
6y38_c
6y3r_p
6y3s_p
6y3v_p
6y40_p
6y4q_c
6y58_p
6y7v_b
6y8e_p

In [74]:
pssm_cc_all = np.array(pssm_cc_all)
pssm_cc_all

array([[-7.73328751e-04,  1.21859426e+00,  1.16918715e+00, ...,
        -3.51950507e-01,  2.86939337e+00,  4.36587042e-01],
       [-7.73328751e-04,  1.21859426e+00,  1.16918715e+00, ...,
        -3.51950507e-01,  2.86939337e+00,  4.36587042e-01],
       [-7.73328751e-04,  1.21859426e+00,  1.16918715e+00, ...,
        -3.51950507e-01,  2.86939337e+00,  4.36587042e-01],
       ...,
       [ 4.27001570e-01,  2.36106750e+00,  2.47095761e+00, ...,
         3.56357928e-01, -4.23861852e-01, -8.63422292e-01],
       [-6.00000000e+00,  9.00000000e+00,  9.00000000e+00, ...,
         7.50000000e-01, -3.75000000e+00, -2.25000000e+00],
       [ 2.50000000e-01,  5.05000000e+00,  4.95000000e+00, ...,
        -2.22222222e-02, -8.22222222e-01, -7.55555556e-01]])

In [75]:
pssm_cc_all_df = pd.DataFrame([x for x in pssm_cc_all])
pssm_cc_all_df['pdb_chain']=pssm_complete['pdb_chain']

In [76]:
pssm_cc_all_df.to_csv('pssm_complete_cc.csv',index=False)
# pssm_ac_all_df.to_csv('pssm_ac.csv',index=False)

## Padding Intrinsic Disorder

In [80]:
import numpy as np
import math
def sigmoid(x):
	return 1 / (1 + math.exp(-x))

sigmoid_array=np.vectorize(sigmoid)

def padding_intrinsic_disorder(x,N):
	padding_array = np.zeros([N,x.shape[1]])
	if x.shape[0]>=N: # sequence is longer than N
		padding_array[:N,:x.shape[1]] = x[:N,:]
	else:
		padding_array[:x.shape[0],:x.shape[1]] = x
	return padding_array

with open('E:/example_data_feature/Intrinsic Disorder/output_intrisic_dict4', 'rb') as f: # value: (sequence_length, 3): long, short, anchor
	peptide_intrinsic_dict = pickle.load(f,encoding='latin1')



In [81]:
peptide_intrinsic_dict

{'1acy_p': array([[0.4051, 0.3983, 0.8823, 0.3983],
        [0.4051, 0.4124, 0.8391, 0.4124],
        [0.4652, 0.4152, 0.8158, 0.4152],
        [0.433 , 0.4186, 0.7458, 0.4186],
        [0.3599, 0.4136, 0.6474, 0.4136],
        [0.2849, 0.406 , 0.5374, 0.406 ],
        [0.3117, 0.4009, 0.4967, 0.4009],
        [0.3392, 0.4036, 0.4458, 0.4036],
        [0.3704, 0.4085, 0.4149, 0.4085],
        [0.3983, 0.4218, 0.3668, 0.4218],
        [0.3529, 0.4219, 0.2657, 0.4219],
        [0.384 , 0.4092, 0.2865, 0.4092],
        [0.374 , 0.3902, 0.2865, 0.3902],
        [0.2988, 0.38  , 0.2167, 0.38  ],
        [0.2609, 0.3664, 0.2483, 0.3664],
        [0.2328, 0.3684, 0.2865, 0.3684],
        [0.1852, 0.3767, 0.3146, 0.3767],
        [0.2328, 0.3784, 0.4282, 0.3784],
        [0.1969, 0.3732, 0.4703, 0.3732],
        [0.2503, 0.3677, 0.5802, 0.3677],
        [0.2364, 0.3546, 0.6374, 0.3546],
        [0.1881, 0.349 , 0.6474, 0.349 ],
        [0.1731, 0.3543, 0.6984, 0.3543],
        [0.1048, 0.3647,

In [82]:
feature_peptide = []
m=0
n=0
seq_list = []
for i in peptide_intrinsic_dict.keys():
    feature_ID = padding_intrinsic_disorder(peptide_intrinsic_dict[i],50)
    feature_peptide.append( feature_ID.flatten())
    seq_list.append(i)
feature_peptide = np.array(feature_peptide)
feature_peptide

array([[0.4051, 0.3983, 0.8823, ..., 0.    , 0.    , 0.    ],
       [0.4766, 0.4187, 0.9103, ..., 0.    , 0.    , 0.    ],
       [0.712 , 0.6661, 0.9562, ..., 0.    , 0.    , 0.    ],
       ...,
       [0.9269, 0.8684, 0.9922, ..., 0.    , 0.    , 0.    ],
       [1.    , 1.    , 1.    , ..., 0.    , 0.    , 0.    ],
       [0.0985, 0.0675, 0.9534, ..., 0.    , 0.    , 0.    ]])

In [83]:
feature_peptide_df = pd.DataFrame(feature_peptide)
feature_peptide_df['pdb_chain']=seq_list
feature_peptide_df.to_csv('IntrinsicD_peptide.csv',index=False)